<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_custom_model_in_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스 API를 사용한 사용자 정의 모델 만들기 with 텐서플로 2.3

DLD(Daejeon Learning Day) 2020을 위해 작성된 노트북입니다.

* 깃허브 주소: https://github.com/rickiepark/handson-ml2/blob/master/custom_model_in_keras.ipynb
* 코랩 주소: https://colab.research.google.com/github/rickiepark/handson-ml2/blob/master/custom_model_in_keras.ipynb

In [ ]:
import tensorflow as tf

tf.__version__

'2.3.0'

### MNIST 손글씨 숫자 데이터 적재

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.reshape(-1, 784) / 255.   # reshape( )으로 784개의 1차원 배열로 변경하고, 전처리(0~1사이의 실수값)으로 ( / 255.) 변경

# 밀집층(완전연결층)이라고 불리는 기본적인 신경연결망의 층을 이용
# MNIST 데이터셋은 28 * 28 의 2차원 배열이지만
# numpy의 reshape( )으로 784개의 1차원 배열로 변경
# 신경망 사용시에는 0 ~ 255의 pixel값을 그대로 사용하는 것 보다는 
# 0 ~ 1 사이의 값(실수값)으로 신경망을 훈련하는 것이 효율적이므로 ==>  / 255. 처리를 해 줌 !!!


## 사용할 데이터 셋 확인

In [ ]:
X_train.shape  # 784개의 원소(pixel)를 확인할 수 있다.

(60000, 784)

## 사용자 정의층을 만들기 전에...
## Keras에서 사용하는 대표적인 2가지 모델을 살펴보자!

### `Sequential()` 클래스와 함수형 API의 관계 ==> 이 두가지를 사용해서 딥러닝 모델을 주로 만든다!

# 1. Sequential( ) 클래스 :: 모델 생성 방법

시퀀셜 모델에 10개의 유닛을 가진 완전 연결 층을 추가합니다.

In [ ]:
seq_model = tf.keras.Sequential()

seq_model.add(tf.keras.layers.Dense(units=10,  # unit(뉴런)의 갯수는 임의로
                                    activation='softmax',
                                    input_shape=(784,)))  # 입력받는 배열의 크기

seq_model.summary()     # 생성된 모델의 구조를 출력할 수 있다.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


결과 :: 모델이름, Dense class, 이 층을 통해서 출력되는 tensor의 크기(Shape), 이 층에서 사용하는 parameter의 갯수(Param #)

## sequential class를 사용한 Keras 신경망!!!

In [ ]:
# 훈련!!!
seq_model.compile(loss='sparse_categorical_crossentropy',   # compile 메소드 : loss(손실함수) 지정
                  metrics=['accuracy'])                     # metrics  : 출력하고 싶은 지표값을 지정
seq_model.fit(X_train, y_train, batch_size=32, epochs=2)    # 훈련보다는 "사용자 정의 모델"을 만드는 것이 목표이므로 
                                                            # epochs는 2만큼만 지정!!  ==>  그래도 accuracy는 91% 정도 나옴..

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4384 - accuracy: 0.8819
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3030 - accuracy: 0.9160


# 2. 함수형 API: 모델 생성 방법

함수형 API를 사용할 때는 `Input()`을 사용해 입력의 크기를 정의해야 합니다. 하지만 `InputLayer` 층이 추가되어 있습니다.

In [ ]:
   # 입력과 출력을 chain처럼 연결함 
   # 모델이라는 class를 사용해서, 이 class에 입력과 출력을 지정해서 Keras 모델을 만든다!!
   ########################
inputs = tf.keras.layers.Input(784)        # 784개의 1차원 배열값을 입력값으로 지정
 
   # 위의 결과값인 inputs값을 Dense층에 입력받아서 아래에서 outputs값이 나오고, 
   # 다음 Dense층을 추가한다면 또 outputs값이 inputs값으로 들어가는 작업이 반복된다.

outputs = tf.keras.layers.Dense(units=10,        
                                activation='softmax')(inputs)  # __call()__ 메서드 호출
# dense = tf.keras.layers.Dense(units=10, activation='softmax')
# outputs = dense(inputs)

func_model = tf.keras.Model(inputs, outputs)    #  모델층을 만들때 inputs, outputs을 지정

func_model.summary()                            #  만들어진 모델의 구조를 출력

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


 결과 : 위에서 본 sequetial 모델과 비슷한 결과이지만 input_1이라는 2차원 배열 layer가 추가되었다,

In [ ]:
  # 역시 훈련
  
func_model.compile(loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
func_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4372 - accuracy: 0.8842
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3017 - accuracy: 0.9157


### `Input`의 정체는 무엇일까요? 이 함수는 `InputLayer` 클래스의 객체를 만들어 그 결과를 반환합니다.

In [ ]:
type(tf.keras.layers.Input)   # input은  함.수!!!

function

## 사실 신경망의 입력층은 입력 그 자체입니다. `InputLayer` 객체의 입력 노드 출력을 그대로 `Dense` 층에 주입할 수 있습니다. 모든 층은 입력과 출력 노드를 정의합니다.

In [ ]:
# inputs = tf.keras.layers.Input(784)

input_layer = tf.keras.layers.InputLayer(784)    # 위의 출력모델에서 보인 InputLayer라는 레이어를 사용해서 코드작성
inputs = input_layer._inbound_nodes[0].outputs   # inputs = tf.keras.layers.Input(784) 명령과 같은 의미

# outputs = tf.keras.layers.Dense(units=10,        
#                                activation='softmax')(inputs) 
outputs = tf.keras.layers.Dense(units=10,
                                activation='softmax')(inputs)

input_layer_model = tf.keras.Model(inputs, outputs)

input_layer_model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
  #  훈련!!

input_layer_model.compile(loss='sparse_categorical_crossentropy', 
                          metrics=['accuracy'])
input_layer_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4406 - accuracy: 0.8827
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3024 - accuracy: 0.9161


### 함수형 API를 사용한 모델은 `layers` 속성에 `InputLayer` 클래스를 포함합니다.

In [ ]:
func_model.layers

### 하지만 시퀀셜 모델은 `layers` 속성에 `InputLayer` 클래스가 보이지 않습니다.

In [ ]:
seq_model.layers

### 숨겨진 layer를 확인하려면  감춰진 `_layers` 속성으로 확인합니다. 여기에서 `InputLayer` 클래스를 확인할 수 있습니다.

In [ ]:
seq_model._layers  # sequencial 모델도 InputLayer가 있지만 숨겨져 있음... 확인해 볼려면!!!

# InputLayer가 만들어지는 시점은.... 처음 Dense층을 만들때 매개변수로 넣은 input_shape값을 가지고 InputLayer 객체를 생성 해 준다!!

################################

### 또는 `_input_layers` 속성에서도 확인할 수 있습니다. :: seauencial모델과 funtion모델(함수형 API) 둘 다 InputLayer가 있음을 확인할 수 있다!!

In [ ]:
seq_model._input_layers, func_model._input_layers

([<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7f81d9ddefd0>],
 [<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7f81d95a8748>])

In [ ]:
seq_model._output_layers, func_model._output_layers

([<tensorflow.python.keras.layers.core.Dense at 0x7f81d9ddea20>],
 [<tensorflow.python.keras.layers.core.Dense at 0x7f81d95a8898>])

### `Model` 클래스로 만든 `func_model`은 사실 `Functional` 클래스의 객체입니다. `Model` 클래스는 서브클래싱에 사용합니다.

In [ ]:
func_model.__class__

tensorflow.python.keras.engine.functional.Functional

### 시퀀셜 모델은 함수형 모델의 특별한 경우입니다. (`Model` --> `Functional` --> `Sequential`)

# 사용자 정의 층 만들기

### `tf.layers.Layer` 클래스를 상속하고 `build()` 메서드에서 가중치를 만든다음 `call()` 메서드에서 연산을 구현합니다.

In [ ]:
class MyDense(tf.keras.layers.Layer):
    
    def __init__(self, units, activation=None, **kwargs):
        # units와 activation 매개변수 외에 나머지 변수를 부모 클래스의 생성자로 전달합니다.
        super(MyDense, self).__init__(**kwargs)
        self.units = units
        # 문자열로 미리 정의된 활성화 함수를 선택합니다. e.g., 'softmax', 'relu'
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        # __call__() 메서드를 호출할 때 호출됩니다. 가중치 생성을 지연합니다.
        # 가중치와 절편을 생성합니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=[input_shape[-1], self.units],
                                      initializer='glorot_uniform'   # 케라스의 기본 초기화
                                     )
        self.bias = self.add_weight(name='bias',
                                    shape=[self.units],
                                    initializer='zeros')
    
    def call(self, inputs):  # training=None은 training은 배치 정규화나 드롭아웃 같은 경우 사용
        # __call__() 메서드를 호출할 때 호출됩니다.
        # 실제 연산을 수행합니다. [batch_size, units]
        z = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            return self.activation(z)
        return z

In [ ]:
inputs = tf.keras.layers.Input(784)

# Layer.__call__() --> MyDense().build() --> Layer.build() --> MyDense().call()

outputs = MyDense(units=10, activation='softmax')(inputs)

my_dense_model = tf.keras.Model(inputs, outputs)

my_dense_model.summary()


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
my_dense_7 (MyDense)         (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
my_dense_model.compile(loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])
my_dense_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4395 - accuracy: 0.8823
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3024 - accuracy: 0.9163


# 사용자 정의 모델 만들기

In [ ]:
# fit(), compile(), predict(), evaluate() 등의 메서드 제공
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.output_layer = MyDense(units=10, activation='softmax')
    
    def call(self, inputs):
        return self.output_layer(inputs)

In [ ]:
my_model = MyModel()

my_model.compile(loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])
my_model.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4419 - accuracy: 0.8808
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3029 - accuracy: 0.9161


# 사용자 정의 훈련

In [ ]:
class MyCustomStep(MyModel):
    
    def train_step(self, data):
        # fit()에서 전달된 데이터
        x, y = data

        # 그레이디언트 기록 시작
        with tf.GradientTape() as tape:
            # 정방향 계산
            y_pred = self(x)
            # compile() 메서드에서 지정한 손실 계산
            loss = self.compiled_loss(y, y_pred)

        # 훈련가능한 파라미터에 대한 그레이디언트 계산
        gradients = tape.gradient(loss, self.trainable_variables)
        # 파라미터 업데이트
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # compile() 메서드에서 지정한 지표 계산
        self.compiled_metrics.update_state(y, y_pred)
        
        # 현재까지 지표와 결괏값을 딕셔너리로 반환
        return {m.name: m.result() for m in self.metrics}

In [ ]:
my_custom_step = MyCustomStep()

my_custom_step.compile(loss='sparse_categorical_crossentropy', 
                       metrics=['accuracy'])
my_custom_step.fit(X_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4378 - accuracy: 0.8832
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3031 - accuracy: 0.9161
